In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.datasets import cifar10  # Change here to load CIFAR-10 instead of CIFAR-100
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset instead of CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)  # Change to 10 classes for CIFAR-10
y_test = to_categorical(y_test, 10)  # Change to 10 classes for CIFAR-10

# Define ResNet-18 architecture
def resnet_block(input_tensor, filters, strides=(1, 1), downsample=False):
    identity = input_tensor
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    if downsample:
        identity = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(identity)
        identity = layers.BatchNormalization()(identity)
    x = layers.add([x, identity])
    x = layers.Activation('relu')(x)
    return x

def resnet18(input_shape=(32, 32, 3), num_classes=10):  # Change num_classes to 10 for CIFAR-10
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = resnet_block(x, 64)
    x = resnet_block(x, 64)
    x = resnet_block(x, 128, downsample=True)
    x = resnet_block(x, 128)
    x = resnet_block(x, 256, downsample=True)
    x = resnet_block(x, 256)
    x = resnet_block(x, 512, downsample=True)
    x = resnet_block(x, 512)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# Create ResNet-18 model
model = resnet18()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


170498071/170498071 [==============================] - 4s 0us/step
Epoch 1/20
782/782 [==============================] - 56s 50ms/step - loss: 1.3681 - accuracy: 0.5070 - val_loss: 1.9680 - val_accuracy: 0.4342
Epoch 2/20
782/782 [==============================] - 39s 50ms/step - loss: 0.9631 - accuracy: 0.6589 - val_loss: 1.5523 - val_accuracy: 0.5073
Epoch 3/20
782/782 [==============================] - 38s 48ms/step - loss: 0.7952 - accuracy: 0.7218 - val_loss: 0.8801 - val_accuracy: 0.6944
Epoch 4/20
782/782 [==============================] - 38s 49ms/step - loss: 0.6759 - accuracy: 0.7640 - val_loss: 2.0250 - val_accuracy: 0.4907
Epoch 5/20
782/782 [==============================] - 38s 49ms/step - loss: 0.5923 - accuracy: 0.7929 - val_loss: 0.9618 - val_accuracy: 0.6862
Epoch 6/20
782/782 [==============================] - 38s 49ms/step - loss: 0.5246 - accuracy: 0.8170 - val_loss: 1.0517 - val_accuracy: 0.6724
Epoch 7/20
782/782 [==============================] - 38s 49ms/step -

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset instead of CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)  # Change to 10 classes for CIFAR-10
y_test = to_categorical(y_test, 10)  # Change to 10 classes for CIFAR-10

# Define SE block
def se_block(input_tensor, ratio=16):
    num_channels = input_tensor.shape[-1]
    squeeze = layers.GlobalAveragePooling2D()(input_tensor)
    excitation = layers.Dense(num_channels // ratio, activation='relu')(squeeze)
    excitation = layers.Dense(num_channels, activation='sigmoid')(excitation)
    excitation = tf.expand_dims(tf.expand_dims(excitation, axis=1), axis=1)
    return layers.multiply([input_tensor, excitation])

# Define ResNet-18 with SE blocks
def resnet18_se(input_shape=(32, 32, 3), num_classes=10):  # Change num_classes to 10 for CIFAR-10
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(512, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.Conv2D(512, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = se_block(x)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# Create ResNet-18 with SE blocks
model = resnet18_se()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/20
782/782 [==============================] - 38s 34ms/step - loss: 1.0978 - accuracy: 0.6021 - val_loss: 1.3879 - val_accuracy: 0.5859
Epoch 2/20
782/782 [==============================] - 25s 32ms/step - loss: 0.6582 - accuracy: 0.7683 - val_loss: 0.8070 - val_accuracy: 0.7246
Epoch 3/20
782/782 [==============================] - 26s 33ms/step - loss: 0.5028 - accuracy: 0.8247 - val_loss: 0.7675 - val_accuracy: 0.7434
Epoch 4/20
782/782 [==============================] - 26s 33ms/step - loss: 0.4001 - accuracy: 0.8614 - val_loss: 0.8164 - val_accuracy: 0.7302
Epoch 5/20
782/782 [==============================] - 25s 32ms/step - loss: 0.3138 - accuracy: 0.8900 - val_loss: 0.6812 - val_accuracy: 0.7875
Epoch 6/20
782/782 [==============================] - 25s 32ms/step - loss: 0.2395 - accuracy: 0.9175 - val_loss: 0.7681 - val_accuracy: 0.7668
Epoch 7/20
782/782 [==============================] - 25s 32ms/step - loss: 0.1839 - accuracy: 0.9341 - val_loss: 0.7429 - val_accuracy:

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset instead of CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)  # Change to 10 classes for CIFAR-10
y_test = to_categorical(y_test, 10)  # Change to 10 classes for CIFAR-10

# Define CNN model without attention mechanism
def simple_cnn():
    inputs = layers.Input(shape=(32, 32, 3))
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)  # Change to 10 classes for CIFAR-10
    model = Model(inputs, outputs)
    return model

# Create the model
model = simple_cnn()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/10
782/782 [==============================] - 9s 9ms/step - loss: 1.3826 - accuracy: 0.4983 - val_loss: 1.0374 - val_accuracy: 0.6259
Epoch 2/10
782/782 [==============================] - 6s 8ms/step - loss: 0.9268 - accuracy: 0.6727 - val_loss: 0.8397 - val_accuracy: 0.7049
Epoch 3/10
782/782 [==============================] - 6s 8ms/step - loss: 0.7278 - accuracy: 0.7453 - val_loss: 0.7884 - val_accuracy: 0.7278
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.5811 - accuracy: 0.7974 - val_loss: 0.8007 - val_accuracy: 0.7272
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.4530 - accuracy: 0.8430 - val_loss: 0.7345 - val_accuracy: 0.7571
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3345 - accuracy: 0.8832 - val_loss: 0.7773 - val_accuracy: 0.7622
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2377 - accuracy: 0.9177 - val_loss: 0.8836 - val_accuracy: 0.7465
Epoch 

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset instead of CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)  # Change to 10 classes for CIFAR-10
y_test = to_categorical(y_test, 10)  # Change to 10 classes for CIFAR-10

# Define SE block
def se_block(input_tensor, ratio=16):
    num_channels = input_tensor.shape[-1]
    squeeze = layers.GlobalAveragePooling2D()(input_tensor)
    excitation = layers.Dense(num_channels // ratio, activation='relu')(squeeze)
    excitation = layers.Dense(num_channels, activation='sigmoid')(excitation)
    excitation = tf.expand_dims(tf.expand_dims(excitation, axis=1), axis=1)
    return layers.multiply([input_tensor, excitation])

# Define CNN model with SE block
def simple_cnn_se():
    inputs = layers.Input(shape=(32, 32, 3))
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = se_block(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = se_block(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = se_block(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)  # Change to 10 classes for CIFAR-10
    model = Model(inputs, outputs)
    return model

# Create the model
model = simple_cnn_se()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/10
782/782 [==============================] - 12s 12ms/step - loss: 1.5174 - accuracy: 0.4465 - val_loss: 1.1971 - val_accuracy: 0.5671
Epoch 2/10
782/782 [==============================] - 9s 11ms/step - loss: 1.0538 - accuracy: 0.6265 - val_loss: 0.9522 - val_accuracy: 0.6601
Epoch 3/10
782/782 [==============================] - 9s 12ms/step - loss: 0.8527 - accuracy: 0.7029 - val_loss: 0.8489 - val_accuracy: 0.7074
Epoch 4/10
782/782 [==============================] - 8s 11ms/step - loss: 0.7116 - accuracy: 0.7533 - val_loss: 0.7549 - val_accuracy: 0.7396
Epoch 5/10
782/782 [==============================] - 9s 11ms/step - loss: 0.5952 - accuracy: 0.7930 - val_loss: 0.7326 - val_accuracy: 0.7489
Epoch 6/10
782/782 [==============================] - 9s 11ms/step - loss: 0.4904 - accuracy: 0.8287 - val_loss: 0.7850 - val_accuracy: 0.7476
Epoch 7/10
782/782 [==============================] - 8s 11ms/step - loss: 0.3883 - accuracy: 0.8644 - val_loss: 0.7717 - val_accuracy: 0.750

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset instead of CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)  # Change to 10 classes for CIFAR-10
y_test = to_categorical(y_test, 10)  # Change to 10 classes for CIFAR-10

# Define MobileNet architecture
def mobile_net(input_shape=(32, 32, 3), num_classes=10):  # Change num_classes to 10 for CIFAR-10
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = depthwise_separable_conv_block(x, 64, (3, 3))
    x = depthwise_separable_conv_block(x, 128, (3, 3), strides=(2, 2))
    x = depthwise_separable_conv_block(x, 128, (3, 3))

    x = depthwise_separable_conv_block(x, 256, (3, 3), strides=(2, 2))
    x = depthwise_separable_conv_block(x, 256, (3, 3))

    x = depthwise_separable_conv_block(x, 512, (3, 3), strides=(2, 2))
    for _ in range(5):
        x = depthwise_separable_conv_block(x, 512, (3, 3))

    x = depthwise_separable_conv_block(x, 1024, (3, 3), strides=(2, 2))
    x = depthwise_separable_conv_block(x, 1024, (3, 3))

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

def depthwise_separable_conv_block(input_tensor, filters, kernel_size, strides=(1, 1)):
    x = layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, (1, 1), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    return x

# Create MobileNet model
model = mobile_net()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/20
782/782 [==============================] - 35s 25ms/step - loss: 1.9091 - accuracy: 0.3012 - val_loss: 1.6926 - val_accuracy: 0.3726
Epoch 2/20
782/782 [==============================] - 18s 23ms/step - loss: 1.5426 - accuracy: 0.4404 - val_loss: 1.8552 - val_accuracy: 0.4173
Epoch 3/20
782/782 [==============================] - 18s 23ms/step - loss: 1.3779 - accuracy: 0.5084 - val_loss: 1.5113 - val_accuracy: 0.4630
Epoch 4/20
782/782 [==============================] - 18s 23ms/step - loss: 1.2471 - accuracy: 0.5571 - val_loss: 2.5551 - val_accuracy: 0.4512
Epoch 5/20
782/782 [==============================] - 19s 24ms/step - loss: 1.1669 - accuracy: 0.5901 - val_loss: 1.2667 - val_accuracy: 0.5771
Epoch 6/20
782/782 [==============================] - 18s 23ms/step - loss: 1.0427 - accuracy: 0.6361 - val_loss: 1.2493 - val_accuracy: 0.5755
Epoch 7/20
782/782 [==============================] - 19s 24ms/step - loss: 0.9818 - accuracy: 0.6587 - val_loss: 1.2216 - val_accuracy:

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 data instead of CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert class vectors to binary class matrices (for use with categorical_crossentropy)
y_train = to_categorical(y_train, 10)  # Change to 10 classes for CIFAR-10
y_test = to_categorical(y_test, 10)  # Change to 10 classes for CIFAR-10

def squeeze_excite_block(input_tensor, ratio=16):
    init = input_tensor
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = layers.GlobalAveragePooling2D()(init)
    se = layers.Reshape(se_shape)(se)
    se = layers.Dense(filters // ratio, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)

    x = layers.multiply([init, se])
    return x

def depthwise_separable_conv_block(input_tensor, filters, kernel_size, strides=(1, 1), include_se=False):
    x = layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, (1, 1), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    if include_se:
        x = squeeze_excite_block(x)

    return x

def MobileNetSE(input_shape=(32, 32, 3), num_classes=10):  # Change num_classes to 10 for CIFAR-10
    inputs = layers.Input(shape=input_shape)

    # Adapted MobileNet architecture with SE blocks
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = depthwise_separable_conv_block(x, 64, (3, 3), include_se=True)
    x = depthwise_separable_conv_block(x, 128, (3, 3), strides=(2, 2), include_se=True)
    x = depthwise_separable_conv_block(x, 128, (3, 3), include_se=True)
    x = depthwise_separable_conv_block(x, 256, (3, 3), strides=(2, 2), include_se=True)
    x = depthwise_separable_conv_block(x, 256, (3, 3), include_se=True)
    x = depthwise_separable_conv_block(x, 512, (3, 3), strides=(2, 2), include_se=True)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# Instantiate and compile the model
model = MobileNetSE()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/20
782/782 [==============================] - 29s 23ms/step - loss: 1.4525 - accuracy: 0.4709 - val_loss: 1.4384 - val_accuracy: 0.4979
Epoch 2/20
782/782 [==============================] - 16s 21ms/step - loss: 1.0800 - accuracy: 0.6143 - val_loss: 1.2301 - val_accuracy: 0.5672
Epoch 3/20
782/782 [==============================] - 16s 20ms/step - loss: 0.8862 - accuracy: 0.6867 - val_loss: 1.0008 - val_accuracy: 0.6502
Epoch 4/20
782/782 [==============================] - 16s 21ms/step - loss: 0.7638 - accuracy: 0.7293 - val_loss: 1.1341 - val_accuracy: 0.6207
Epoch 5/20
782/782 [==============================] - 16s 20ms/step - loss: 0.6648 - accuracy: 0.7654 - val_loss: 0.9690 - val_accuracy: 0.6793
Epoch 6/20
782/782 [==============================] - 16s 20ms/step - loss: 0.5846 - accuracy: 0.7951 - val_loss: 0.9469 - val_accuracy: 0.6886
Epoch 7/20
782/782 [==============================] - 17s 21ms/step - loss: 0.5229 - accuracy: 0.8138 - val_loss: 0.9228 - val_accuracy: